In [61]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from spektral.layers import GCNConv
from spektral.data import Graph, Dataset
from spektral.data.loaders import DisjointLoader
from spektral.transforms import LayerPreprocess

In [62]:
# Parameters
num_graphs = 1000
min_nodes = 5
max_nodes = 15
node_feature_ranges = [(0, 5000), (0, 10)]
edge_feature_ranges = [(0, 30), (-50, 50)]


In [63]:
# Function to generate a random graph
def generate_random_graph():
    n_nodes = np.random.randint(min_nodes, max_nodes + 1)
    node_features = np.column_stack([
        np.random.uniform(low, high, n_nodes) for low, high in node_feature_ranges
    ])
    adj_matrix = np.random.randint(0, 2, (n_nodes, n_nodes)).astype(np.float32)
    np.fill_diagonal(adj_matrix, 0)  # No self-loops

    # Generate edge features
    edge_features = []
    for i in range(n_nodes):
        for j in range(i + 1, n_nodes):
            if adj_matrix[i, j] > 0:
                edge_feature = np.array([
                    np.random.uniform(low, high) for low, high in edge_feature_ranges
                ])
                edge_features.append(edge_feature)
    edge_features = np.array(edge_features)

    return Graph(x=node_features, a=adj_matrix, e=edge_features)

In [64]:
# Create a dataset using Spektral's data structure
class MyGraphDataset(Dataset):
    def read(self):
        return [generate_random_graph() for _ in range(num_graphs)]

In [65]:
# Define the GCN Model
class EdgeFeatureGCN(Model):
    def __init__(self, n_hidden, n_edge_features):
        super().__init__()
        
        # GCN layers to process node features
        self.gcn1 = GCNConv(n_hidden, activation='relu')
        self.gcn2 = GCNConv(n_hidden, activation='relu')
        
        # Dense layer for edge feature prediction (MLP for edge features)
        self.edge_predictor = Dense(n_edge_features, activation='linear')

    def build(self, input_shape):
        self.gcn1.build(input_shape)
        self.gcn2.build(input_shape)
        self.edge_predictor.build((input_shape[0][0], input_shape[0][1] * 2))
        super().build(input_shape)

    def call(self, inputs):
        x, a = inputs  # Node features (x) and adjacency matrix (a)
        
        # Ensure no None values
        if x is None or a is None:
            raise ValueError("Node features and adjacency matrix must not be None")
        
        # Ensure adjacency matrix is float32
        a = tf.cast(a, tf.float32)
        
        # GCN layers
        h = self.gcn1([x, a])
        h = self.gcn2([h, a])
        
        # Predict edge features using node embeddings
        edge_features = []
        for i in range(x.shape[0]):
            for j in range(i + 1, x.shape[0]):  # Iterate over node pairs
                if a[i, j] > 0:  # Only consider connected nodes
                    # Concatenate or take difference between node embeddings
                    node_pair_embedding = tf.concat([h[i], h[j]], axis=-1)
                    
                    # Predict edge feature for node pair (i, j)
                    predicted_edge_feature = self.edge_predictor(node_pair_embedding)
                    edge_features.append(predicted_edge_feature)

        return tf.stack(edge_features)

In [66]:

# Instantiate and compile the model
n_hidden = 16  # Number of hidden units in GCN
n_edge_features = 2  # Number of edge features to predict
model = EdgeFeatureGCN(n_hidden=n_hidden, n_edge_features=n_edge_features)



In [67]:

# Compile the model (assuming you have the ground truth edge features for training)
model.compile(optimizer='adam', loss='mse')


In [68]:

# Prepare data loader
dataset = MyGraphDataset()
loader = DisjointLoader(dataset)

In [70]:
# Modify the loader to return data in the format (x, y)
def format_loader(loader):
    for batch in loader:
        x, a, e, i = batch
        yield (x, a), e


In [72]:

# Train the model (for demonstration purposes)
model.fit(format_loader(loader), epochs=10)

Epoch 1/10


/home/philippe/anaconda3/envs/Master/lib/python3.12/site-packages/spektral/data/utils.py:221: UserWarning: you are shuffling a 'MyGraphDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


ValueError: Exception encountered when calling GCNConv.call().

[1mTried to convert 'y' to a tensor and failed. Error: None values not supported.[0m

Arguments received by GCNConv.call():
  • inputs=['tf.Tensor(shape=(None, 2), dtype=float32)', 'tf.Tensor(shape=(None, None), dtype=float32)']
  • mask=['None', 'None']